In [20]:
!pip install transformers[torch]
!pip install small-text
!pip install --upgrade datasets==4.1.0
!pip install numpy
!pip install scikit-learn
!pip install matplotlib seaborn


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import datasets, torch
import numpy as np

seed= 2024
torch.manual_seed(seed)
np.random.seed(seed)

Dataset Preparation

In [73]:
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names

ds_path= "cornell-movie-review-data/rotten_tomatoes"
ds_builder= load_dataset_builder(ds_path)
tomatoes_ds= load_dataset(ds_path)



In [76]:
print(ds_builder.info.description)
print(ds_builder.info.features)
print(get_dataset_split_names(ds_path))



{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}
['train', 'validation', 'test']


In [78]:
print(tomatoes_ds['train'][:1])

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'], 'label': [1]}


In [83]:
from datasets import DatasetDict
# On the train splited dataset.
# 80% for training. 10% for validation. 10% for test
train_test_val= tomatoes_ds['train'].train_test_split(test_size= .2)
test_val= train_test_val['test'].train_test_split(test_size= .5)

# Putting into a single DatasetDict
dataset= DatasetDict({
  "train": train_test_val['train'],
  "test": test_val['test'],
  "validation": test_val['train'],
})

In [86]:
print(dataset.num_columns)
print(dataset.num_rows)
print(dataset['train'][0])
print(dataset['validation'][0])
print(dataset['test'][0])

{'train': 2, 'test': 2, 'validation': 2}
{'train': 6824, 'test': 853, 'validation': 853}
{'text': 'a classy item by a legend who may have nothing left to prove but still has the chops and drive to show how its done .', 'label': 1}
{'text': 'the unique tug-of-war with viewer expectations is undeniable , if not a pleasure in its own right .', 'label': 1}
{'text': 'it is a likable story , told with competence .', 'label': 1}


In [88]:
## Import AutoTokenizer to transform raw text into numberical tokens
## Load the pre-trained model
from transformers import AutoTokenizer
model= "bert-base-cased"
tokenizer= AutoTokenizer.from_pretrained(model)


d:\LONGSON\PROJECTS\demo_active_learning\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Cao-IT\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [94]:
num_classes= dataset['train'].features["label"]
print(num_classes)
num_classes= num_classes.num_classes
print(type(num_classes), num_classes)

ClassLabel(names=['neg', 'pos'])
<class 'int'> 2


In [97]:
print(dataset['train'])
print(dataset['train']['text'])
print(dataset['train']['label'])

Dataset({
    features: ['text', 'label'],
    num_rows: 6824
})
Column(['a classy item by a legend who may have nothing left to prove but still has the chops and drive to show how its done .', 'bogdanovich taps deep into the hearst mystique , entertainingly reenacting a historic scandal .', 'spare yet audacious . . .', "one sloughs one's way through the mire of this alleged psychological thriller in search of purpose or even a plot .", "it's a lovely , eerie film that casts an odd , rapt spell ."])
Column([1, 1, 1, 0, 1])
